In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from skimage.io import imread
import tifffile as tf
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
def get_label(i, coords, labels):
    return labels[coords[i, 0], coords[i, 1]]

def get_df_label(i, coords, labels):
    return labels[coords.loc[i, "x"], coords.loc[i, "y"]]

In [3]:
def assign_dots_to_nuclei(root_path, magnification, image_type, mask_type, nfields, suffix, nthreads=1):
    for series_id in range(1, nfields+1):
        dots_data = pd.read_csv(os.path.join(root_path, f"{magnification}_{image_type}_calc", f"{series_id:03d}_allDots_snr2.csv"))
        labels = imread(os.path.join("../../data", f"{magnification}_mask", mask_type, f"a647_{series_id:03d}{suffix}.tiff"))
        xycoords = np.array(np.floor(dots_data.loc[np.logical_and(np.isfinite(dots_data["x"]), np.isfinite(dots_data["y"])), ("x", "y")]).astype("i"))
        dots_data.loc[np.logical_and(np.isfinite(dots_data["x"]), np.isfinite(dots_data["y"])), "nid"] = Parallel(n_jobs=nthreads, verbose=0)(
            delayed(get_label)(i, xycoords, labels) for i in tqdm(range(xycoords.shape[0])))
        dots_data.to_csv(os.path.join(root_path, f"{magnification}_{image_type}_calc", f"{series_id:03d}_allDots_snr2.{mask_type}.csv"))

In [4]:
root_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/"
nthreads = 10

In [5]:
assign_dots_to_nuclei(root_path, "60x", "dw", "dilated_labels_watershed", 7, ".dilated_labels", nthreads)

100%|██████████| 1000000/1000000 [00:32<00:00, 31106.45it/s]


In [6]:
assign_dots_to_nuclei(root_path, "60x", "raw", "dilated_labels_watershed", 7, ".dilated_labels", nthreads)

100%|██████████| 1000000/1000000 [00:26<00:00, 37545.58it/s]


In [8]:
assign_dots_to_nuclei(root_path, "20x", "dw", "dilated_labels_watershed_from60x", 7, ".dilated_labels.from_60x", nthreads)

100%|██████████| 862468/862468 [00:25<00:00, 33409.59it/s]


In [9]:
assign_dots_to_nuclei(root_path, "20x", "raw", "dilated_labels_watershed_from60x", 7, ".dilated_labels.from_60x", nthreads)

100%|██████████| 1000000/1000000 [00:30<00:00, 32579.61it/s]
